In [3]:
import numpy as np
import cv2
from skimage import img_as_float


# Function to calculate intuitionistic fuzzy membership
def intuitionistic_fuzzy_membership(image, a=0.5, b=0.3):
    # Normalize the image
    norm_image = img_as_float(image)

    # Membership function (mu) - fuzzifying the image pixel values
    mu = np.clip(norm_image, 0, 1)

    # Non-membership function (nu) - measuring how much a pixel does not belong to a set
    nu = 1 - mu

    # Hesitation degree (pi) - how uncertain we are about the pixel belonging to a set
    pi = 1 - mu - nu

    return mu, nu, pi


# Function to fuse two intuitionistic fuzzy sets
def fuse_ifs(mu1, nu1, pi1, mu2, nu2, pi2, alpha=0.5):
    # Weighted fusion of membership, non-membership, and hesitation
    fused_mu = alpha * mu1 + (1 - alpha) * mu2
    fused_nu = alpha * nu1 + (1 - alpha) * nu2
    fused_pi = alpha * pi1 + (1 - alpha) * pi2

    # Normalizing to ensure values lie between 0 and 1
    fused_image = np.clip(fused_mu - fused_nu, 0, 1)
    return fused_image


# Medical image fusion algorithm using IFS
def medical_image_fusion(image1, image2, alpha=0.5):
    # Convert images to the YCbCr color space for better luminance processing
    image1_ycbcr = cv2.cvtColor(image1, cv2.COLOR_BGR2YCrCb)
    image2_ycbcr = cv2.cvtColor(image2, cv2.COLOR_BGR2YCrCb)

    # Extract luminance channel (Y)
    image1_Y = image1_ycbcr[:, :, 0]
    image2_Y = image2_ycbcr[:, :, 0]

    # Apply intuitionistic fuzzy membership functions
    mu1, nu1, pi1 = intuitionistic_fuzzy_membership(image1_Y)
    mu2, nu2, pi2 = intuitionistic_fuzzy_membership(image2_Y)

    # Fuse the two images using IFS
    fused_Y = fuse_ifs(mu1, nu1, pi1, mu2, nu2, pi2, alpha)

    # Replace the luminance channel with the fused result
    fused_image_ycbcr = image1_ycbcr.copy()
    fused_image_ycbcr[:, :, 0] = np.uint8(fused_Y * 255)

    # Convert back to RGB color space
    fused_image = cv2.cvtColor(fused_image_ycbcr, cv2.COLOR_YCrCb2BGR)

    return fused_image


# Load two medical RGB images
image1 = cv2.imread("data/fusion/052 (2).png")  # Replace with your medical image path
image2 = cv2.imread("data/fusion/052.png")  # Replace with your medical image path

# Perform fusion
fused_image = medical_image_fusion(image1, image2, alpha=0.1)

# Save the result
cv2.imwrite("fused_image_fuzzy.png", fused_image)

# Display the fused image
# cv2.imshow("Fused Image1", fused_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

True